In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as la
import sklearn.decomposition as skd
import scipy.io                 as sio

<H2>My implementation of PCA:</H2>

- Inputs
     - D -> a centered data matrix, in particular one that has samples across columns and dimensions across the rows
     - numComponents -> how many principal components we want to keep, my implentation is simple and doesn't look at any error metrics, simply keeps numComponents of the PC's and drops the rest blindly
- Outputs
     - eigenVectors -> a matrix which has the same number of rows as the input data matrix has features and numComponents number of columns
- Notes
     - I use SVD to do the eigen value decomposition
     - https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html
     - https://stackoverflow.com/a/12273032

In [3]:
def pca( D, k_or_tol ):
    # Author: C. Howard
    # method to perform PCA on some input data given the dimensionality
    # we want our resulting features to have or given a tolerance on the
    # ratio between singular values and the max singular value

    # perform economy SVD on input data
    [U,S,Vt] = la.svd(D, full_matrices = False )

    # make copy of singular values
    iv = np.copy(S)

    # define current number of weight terms as follow
    k = k_or_tol

    # if we are picking k using a relative difference
    # between singular value magnitudes
    if k_or_tol < 1:
        b = iv >= S[0]*k_or_tol  # find indices for all singular value ratios > threshold
        k = np.sum(b)

    # compute weight matrix and pseudo-inverse of weight matrix using truncated SVD terms
    W       = U[:, :k].T
    Winv    = U[:, :k]

    # return weights and original singular values
    return (W,Winv,S)

<H2>My implementation of ICA:</H2>

- Inputs
     - whitenedData -> a centered and whitened data matrix, I pass the centered data projected onto principal components here
     - alpha -> the learning rate, default is 0.1
     - threshold -> this is what our delta wc needs to be to be considered optimized, default is 1e-7
     - maxSteps -> for every component, we don't spend more than this amount of steps, default is 5000
- Outputs
     - W -> the inverse of the mixing matrix
- Notes
     - Sources:
         - https://github.com/akcarsten/Independent_Component_Analysis
         - https://en.wikipedia.org/wiki/FastICA

In [4]:
def g(u):
    return np.tanh(u)
def gPrime(u):
    return 1 - np.square(np.tanh(u))

In [19]:
def fastIca(signals,  alpha = 1, thresh=1e-8, iterations=5000):
    m, n = signals.shape

    # Initialize random weights
    W = np.random.rand(m, m)

    for c in range(m):
        w = W[c, :].copy().reshape(m, 1)
        w = w / np.sqrt((w ** 2).sum())

        i = 0
        lim = 100
        while ((lim > thresh) & (i < iterations)):

            # Dot product of weight and signal
            ws = np.dot(w.T, signals)

            # Pass w*s into contrast function g
            wg = np.tanh(ws * alpha).T

            # Pass w*s into g prime
            wg_ = (1 - np.square(np.tanh(ws))) * alpha

            # Update weights
            wNew = (signals * wg.T).mean(axis=1) - 0.001*wg_.mean() * w.squeeze()

            # Decorrelate weights              
            wNew = wNew - np.dot(np.dot(wNew, W[:c].T), W[:c])
            wNew = wNew / np.sqrt((wNew ** 2).sum())

            # Calculate limit condition
            lim = np.abs(np.abs((wNew * w).sum()) - 1)

            # Update weights
            w = wNew

            # Update counter
            i += 1

        W[c, :] = w.T
    return W

In [20]:
# load data and set some helper variables
D = np.load("../Data/digits-labels.npz")["d"]
(numRawFeatures, num_images) = D.shape
num_features = 36

In [21]:
# compute mean of images
mu_i = np.matmul(D, np.ones((num_images, 1)) / num_images)

# subtract mean from data
Dn = D - mu_i

# Perform PCA from zero-mean form of data
W1, W1pi, sv = pca(Dn, num_features)

In [22]:

# perform ICA
(Minv, M) = ica(W1, random_seed=17, eps=1e-8)
W2 = np.matmul(Minv, W1)
print(np.shape(W))
W2pi = np.linalg.pinv(W2)
print(np.shape(W2pi))

0


KeyboardInterrupt: 

In [ ]:
# # perform NMF
# nmf_obj = skd.NMF(n_components=num_features, random_state=17)
# H = nmf_obj.fit_transform(np.abs(Dn).T).T
# W3 = nmf_obj.components_.T

In [ ]:
# plot pca features
fig, axs = plt.subplots(6, 6)

for i in range(6):
    for j in range(6):
        dataIndex = 6*i+j
        axs[i, j].imshow(np.reshape(W1pi[:,dataIndex],(28,28),'F'), cmap = plt.cm.gray)
counter = 0
for ax in axs.flatten():
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])
    ax.title.set_text('PC'+ str(counter+1))
    counter += 1
fig.set_figheight(15)
fig.set_figwidth(15)  

In [ ]:

# plot ica features
fig, axs = plt.subplots(6, 6)

for i in range(6):
    for j in range(6):
        dataIndex = 6*i+j
        axs[i, j].imshow(np.reshape(W2pi[:,dataIndex],(28,28),'F'))
counter = 0
for ax in axs.flatten():
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])
    ax.title.set_text('IC'+ str(counter+1))
    counter += 1
fig.set_figheight(15)
fig.set_figwidth(15) 

In [ ]:
print(np.shape(W3))
print(np.shape(H))
fig, axs = plt.subplots(6, 6)

for i in range(6):
    for j in range(6):
        dataIndex = 6*i+j
        axs[i, j].imshow(np.reshape(W3[:,dataIndex],(28,28),'F'), cmap = plt.cm.gray)
counter = 0
for ax in axs.flatten():
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])
    ax.title.set_text('NMF W'+ str(counter+1))
    counter += 1
fig.set_figheight(15)
fig.set_figwidth(15) 